In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
from tqdm import tqdm
from dasr.asr import Wav2VecEnv
from dasr.datasets import get_common_voice
from dasr.datasets import RandomNoiseSNR

In [3]:
def get_common_voice_datasets(train_split="train",
                              test_split="test",
                              desire_snr_db=10):
    
    common_voice = get_common_voice(train_split=train_split,
                                    test_split=test_split)
    
    train_dataset = RandomNoiseSNR(common_voice["train"], desire_snr_db=desire_snr_db)
    test_dataset = RandomNoiseSNR(common_voice["test"], desire_snr_db=desire_snr_db)
    
    return train_dataset, test_dataset

In [4]:
device = "cuda"
path_model = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"

# train_split = "train[0:3000]"
# test_split = "test[0:300]"

train_split = "train"
test_split = "test"

asr = Wav2VecEnv(path_model=path_model, device=device)

train_dataset, test_dataset = get_common_voice_datasets(train_split=train_split,
                                                        test_split=test_split)

In [5]:
def inference_dataset(dataset, asr):
    transcriptions = []
    gts = []

    for i in tqdm(dataset):
        wav = i["noise_audio"].to(device).unsqueeze(0)
        transcription = asr.transcribe(wav)    
        gt = i["transcription"]
        
        transcriptions.extend(transcription)
        gts.append(gt)
        
    df = pd.DataFrame({"transcription": transcriptions, "gt": gts})
    return df

In [6]:
train_df = inference_dataset(train_dataset, asr)
test_df = inference_dataset(test_dataset, asr)

  0%|          | 0/22862 [00:00<?, ?it/s]

100%|██████████| 9630/9630 [06:17<00:00, 25.49it/s]


In [7]:
train_df.head()

,transcription,gt
0,демократия неумально подвегается пафу и арабск...,"Демократия неумолимо продвигается по Африке, и..."
1,доклад международного одетла папа на мить,Доклад Международного агентства по атомной эне...
2,мы разъясняем иму их права и законы,Мы разъясняем им их права и законы.
3,только что завершинлось председательство литы ...,Только что завершилось председательство Литвы ...
4,ведь буфет до последнего человека разом убоили...,Весь буфет до последнего человека разом ввалил...


In [8]:
test_df.head()

,transcription,gt
0,к сожалению эти предложения ненаспе опражения ...,"К сожалению, эти предложения не нашли отражени..."
1,если не будет возражений я буду считать что ас...,"Если не будет возражений, я буду считать, что ..."
2,на ваш афке кмилый город,Новошахтинск — милый город
3,цёдарадо ознуть что числе теряюшихся от поечё ...,"Мы особенно рады отметить, что число скрывающи..."
4,контролюр,Контроллер


In [10]:
len(train_df), len(test_df)

(22862, 9630)

In [9]:
train_df.to_csv("train_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)